In [1]:
# license: GPL v3
# authors: Michael Olesik, Piotr Bartman, Sylwester Arabas
# copyright: Jagiellonian University
# based on Fig. 2 from Yang et al. 2018 (http://dx.doi.org/10.5194/acp-18-7313-2018)

In [2]:
# make local files accessible to import statements
import sys, os
sys.path.insert(0, os.path.join(os.getcwd(), '../..'))
from PySDM_tests.smoke_tests.utils import bdf

In [3]:
from PySDM_examples.Yang_et_al_2018_Fig_2.example import Simulation
from PySDM_examples.Yang_et_al_2018_Fig_2.setup import Setup
from PySDM.simulation.physics import formulae as phys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update({'font.size': 16})
from PySDM_examples.utils.show_plot import show_plot

In [4]:
import pint
si = pint.UnitRegistry()
si.setup_matplotlib()

In [5]:
ix51 = 40
ix503 = 99

In [6]:
schemes = ['default','non_adaptive', 'BDF']
schemes=[schemes[2]]

In [7]:
def runner(scheme='BDF'):
    assert scheme in ['default','non_adaptive','BDF']
    setup = Setup(dt_output = 10*si.second)
    if scheme == 'non_adaptive':
        setup.adaptive = False
        setup.dt_max = 1 #* si.second
    
    assert round(setup.r_dry[  ix51]*1e9, 0) == 51
    assert round(setup.r_dry[ ix503]*1e9, 0) == 503

    # setup.rtol_thd = 1e-7
    # setup.rtol_lnv = 1e-7
    # setup.dt_max = .5

    simulation = Simulation(setup)
    if scheme == 'BDF':
        bdf.patch_particles(simulation.particles, setup.coord, rtol=1e-4)

    output = simulation.run()
    # TODO: print something from simulation.particles.stats.wall_times
    result = {}
    result['r'] = np.array(output["r"]).T * si.metres
    result['t'] = output["t"] * si.seconds
    result['z'] = output["z"] * si.metres
    result['S'] = np.array(output["S"]) * si.dimensionless
    result['q'] = (setup.q0 - np.array(output["qv"])) * si.kilogram / si.kilogram
    result['T'] = np.array(output["T"])
    result['n'] = setup.n / (setup.mass_of_dry_air  * si.kilogram)
    
    arg_T = result['T'].reshape(-1,1).repeat(len(result['n']), axis = 1)
    result['r_cr'] = phys.r_cr(setup.kappa, setup.r_dry, arg_T).transpose()
    return  result

In [8]:
outputs = []
for scheme in schemes:
    outputs.append(runner(scheme))

In [16]:
figsize = (14,9*len(outputs))

In [17]:
fig, ax = plt.subplots(len(schemes), 3, sharey=True, figsize=figsize)
if len(outputs)==1:
    ax = np.array(ax).reshape(len(outputs),3)
for i, data in enumerate(outputs):
    
    ax[i,1].set_title('Scheme: '+f'({schemes[i]})')
    
    ax[i,0].set_ylim([800, 1300])
    ax[i,0].plot(data['q'], data['z'], label="q$_l$")
    ax[i,0].xaxis.set_units(si.gram / si.kilogram)
    ax[i,0].grid()
    ax[i,0].legend(loc='best')

    ax[i,1].plot(data['S']+1, data['z'], label="S+1")
    ax[i,1].grid()
    ax[i,1].legend(loc='best')
    ax[i,1].set_xlim([0.997, 1.003])

    ax[i,2].plot(data['r'][ix51], data['z'], label="r$_d$ = 51 nm")
    ax[i,2].plot(data['r'][ix503], data['z'], label="r$_d$ = 501 nm")
    ax[i,2].grid()
    ax[i,2].xaxis.set_units(si.micrometre)
    ax[i,2].legend(loc='best')
    plt.tight_layout()
    fig.subplots_adjust(top=0.88)

    

show_plot(filename='q_S_rd.pdf')

In [18]:
def rmean(r, n, mask):
    nt = r.shape[1]
    n_dot_r = n.magnitude.dot(np.where(mask, r.magnitude, 0))
    n_tot = np.sum(np.where(mask, n.magnitude.reshape(-1,1).repeat(nt, axis=1), 0), axis=0)
    rmean = np.full(nt, np.nan)
    nmask = n_tot > 0
    rmean[nmask] = n_dot_r[nmask] / n_tot[nmask]
    return rmean * r.units

In [23]:
mgn = lambda value, unit: (value / unit).to_base_units().magnitude


fig, ax = plt.subplots(len(schemes), 1, sharex=True, figsize=figsize)
if len(outputs)==1:
    ax = (ax,)
for i, data in enumerate(outputs):
    
    hist, xedges, yedges = np.histogram2d(
        data['t'].magnitude.reshape(1,-1).repeat(data['r'].shape[0], axis=0).ravel(), 
        data['r'].magnitude.ravel(),
        bins = [figsize[0] * 15, figsize[1] * 10], 
        weights = data['n'].magnitude.reshape(-1,1).repeat(data['r'].shape[1], axis=1).ravel(),
    )
    
    xunit = si.hour
    yunit = si.micrometres
    c = ax[i].pcolormesh(
        mgn(xedges * data['t'].units, xunit), 
        mgn(yedges * data['r'].units, yunit), 
        hist.T,
        cmap = 'coolwarm',
        norm = mpl.colors.LogNorm()
    )
    ax[i].set_title('Scheme: '+f'({schemes[i]})')
    ax[i].yaxis.set_units(yunit)
    ax[i].xaxis.set_units(xunit)
    ax[i].set_ylim([0, 20])

    ax[i].plot(data['t'], rmean(data['r'], data['n'], data['r'].magnitude > data['r_cr']), label="r_mean (r > r_cr)", color='black')
    ax[i].plot(data['t'], rmean(data['r'], data['n'], data['r'] > 1 * si.micrometre), label="r_mean (r > 1 um)", linestyle='--', color='gray')
    ax[i].legend(loc='best')
    ax[i].grid()
    
    plt.tight_layout()
    fig.subplots_adjust(top=0.88)
show_plot(filename='spectrum.pdf')

In [27]:
def n_tot(n, condition):
    return np.dot(n, condition)

In [86]:
fig, ax = plt.subplots(len(schemes),1,figsize=figsize, sharex= True)
if len(outputs)==1:
    ax = (ax,)
xunit = si.hour
yunit = 1/si.microgram
t0 = 1200 * si.second
f0 = 1 / 1000 * si.hertz  # TODO !!!!
for i, data in enumerate(outputs):
    ax[i].set_title('Total number concentration '+f'({schemes[i]})')
    ax[i].yaxis.set_units(yunit)
    ax[i].xaxis.set_units(xunit)
    ax[i].plot(data['t'], n_tot(data['n'], data['r'].magnitude < data['r_cr']), label='not activated (r < r$_{cr}$)', color='blue')
    ax[i].plot(data['t'], n_tot(data['n'], data['r'].magnitude > data['r_cr']), label="n_tot (r > r$_{cr}$)", color='black')
    ax[i].plot(data['t'], n_tot(data['n'], data['r'] > 1 * si.micrometre), label="n_tot (r > 1 $\mu m$)", linestyle='--', color='gray')
    ax[i].set_xticks(t0 + 1/2/f0*np.arange(25), minor=True)
    ax2 = ax[i].twinx()
    ax2.plot(data['t'], data['S']+1, label = 'supersaturation')
    ax2.xaxis.set_units(xunit)
    ax2.set_ylim([.995,1.005])
    ax2.grid(linestyle='--')
    ax[i].legend(loc='upper center')
    ax[i].grid(linestyle='--', which='minor')
    plt.tight_layout()
    fig.subplots_adjust(top=0.88)
show_plot(filename='total_number+S_bdf.pdf')